<a href="https://colab.research.google.com/github/ryrynbob/ust-deep-learning-2026/blob/main/AssignmentWeek4_Ryan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4

## Ryan Nguyen

## Special Topics

## Imports and Device Info

In [18]:
import time
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split



## Create Synthetic Data

In [25]:
# 1. CREATE SYNTHETIC DATA
# ======================
samples = 5000
seq_length = 60
channels = 4
num_classes = 3

## Split Data

In [26]:
X = np.random.rand(samples, seq_length, channels).astype("float32")
y = np.random.randint(0, num_classes, size=(samples,))

# Split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


## Build Splice CNN Model

In [27]:
# 2. BUILD SPLICEFINDER-STYLE CNN
# ======================
def build_model():
    model = keras.Sequential([
        layers.Input(shape=(seq_length, channels)),
        layers.Conv1D(filters=50, kernel_size=7, activation='relu', padding='same'),
        layers.MaxPooling1D(pool_size=2),
        layers.Conv1D(filters=50, kernel_size=7, activation='relu', padding='same'),
        layers.MaxPooling1D(pool_size=2),
        layers.Flatten(),
        layers.Dense(100, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

## Early Stopping Callback

In [28]:
# ======================
# 3. EARLY STOPPING CALLBACK
# ======================
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

## Train on CPU

In [29]:
# 4. TRAIN ON CPU
# ======================
print("\n" + "="*50)
print("TRAINING ON CPU")
print("="*50)

tf.keras.backend.clear_session()
with tf.device('/CPU:0'):
    model_cpu = build_model()
    start = time.time()
    history_cpu = model_cpu.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=32,
        callbacks=[early_stop],
        verbose=1
    )
    cpu_time = time.time() - start

cpu_epochs = len(history_cpu.history['loss'])



TRAINING ON CPU
Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.3431 - loss: 1.1052 - val_accuracy: 0.3360 - val_loss: 1.1057
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.3366 - loss: 1.1023 - val_accuracy: 0.3360 - val_loss: 1.1063
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.3370 - loss: 1.1017 - val_accuracy: 0.3360 - val_loss: 1.1112
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.3362 - loss: 1.1009 - val_accuracy: 0.3360 - val_loss: 1.1032
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.3429 - loss: 1.1001 - val_accuracy: 0.3370 - val_loss: 1.1002
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.3382 - loss: 1.0987 - val_accuracy: 0.3360 - val_loss: 1.1099
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.3532 - loss: 1.0971 - val_accuracy: 0.3360 - val_loss: 1.1211
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.3710 - loss: 

## Train on GPU

In [30]:
# ======================
# 5. TRAIN ON GPU (if available)
# ======================
gpu_time = None
gpu_epochs = None

if tf.config.list_physical_devices('GPU'):
    print("\n" + "="*50)
    print("TRAINING ON GPU")
    print("="*50)

    tf.keras.backend.clear_session()
    with tf.device('/GPU:0'):
        model_gpu = build_model()
        start = time.time()
        history_gpu = model_gpu.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=50,
            batch_size=32,
            callbacks=[early_stop],
            verbose=1
        )
        gpu_time = time.time() - start

    gpu_epochs = len(history_gpu.history['loss'])
else:
    print("\nNo GPU detected. Skipping GPU training.")


TRAINING ON GPU
Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.3515 - loss: 1.1024 - val_accuracy: 0.3360 - val_loss: 1.1014
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3318 - loss: 1.1007 - val_accuracy: 0.3360 - val_loss: 1.1173
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3387 - loss: 1.1020 - val_accuracy: 0.3360 - val_loss: 1.1027
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3542 - loss: 1.0992 - val_accuracy: 0.3360 - val_loss: 1.1194


## Results Comparison

In [31]:
# ======================
# 6. RESULTS COMPARISON
# ======================
print("\n" + "="*50)
print("RESULTS SUMMARY")
print("="*50)
print(f"CPU Training Time: {cpu_time:.2f} seconds ({cpu_epochs} epochs)")
if gpu_time:
    print(f"GPU Training Time: {gpu_time:.2f} seconds ({gpu_epochs} epochs)")
    speedup = cpu_time / gpu_time
    print(f"Speedup (GPU vs CPU): {speedup:.2f}x")
    print(f"GPU is {speedup:.2f} times faster than CPU")
else:
    print("GPU: Not available")


RESULTS SUMMARY
CPU Training Time: 21.81 seconds (8 epochs)
GPU Training Time: 8.99 seconds (4 epochs)
Speedup (GPU vs CPU): 2.43x
GPU is 2.43 times faster than CPU
